Having issues using qiime2 cutadapt for certain sequences and so we had to resort to using the actual cutadapt tool on our fastq.gz sequences...
Might actually have to write a python for-loop for this portion...

But first! We will have to unzip all of our fastq.gz files and make a new directory for our 
fastq files. 

In [1]:
gunzip -k -f 180612_BVDJ2/*.gz
mkdir unzipped_GH17-seqs/
mv /180612_BVDJ2/*.fastq unzipped_GH17-seqs/

SyntaxError: invalid syntax (<ipython-input-1-c539394c1258>, line 1)

Solution: use cutadapt tool outside of qiime2. Because we have to input R1 and R2 files seperately for dada2 have to input samples not as paired end. For example: 

In [ ]:
#for removing 16S samples from our R1 sample (926r, primer)
cutadapt -a CCGYCAATTYMTTTRAGTT \
--match-read-wildcards \
--discard-trimmed \
-m 1 \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R1_001.fastq > /home/qiime2/Greenhouse_experiment_2017/trimmed_ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

In [ ]:
#for removing 16S samples from our R2 sample (F515, primer)
cutadapt -a GTGCCAGCMGCCGCGGTAA \
--match-read-wildcards \
-m 1 \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R2_001.fastq > /home/qiime2/Greenhouse_experiment_2017/trimmed_ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

In [ ]:
#for removing ITS samples from our R1 sample (ITS4 primer)
cutadapt -a TCCTCCGCTTATTGATATGC \
--match-read-wildcards \
-m 1 \
/home/qiime2/Greenhouse_experiment_2017/180612_BVDJ2/Gh_expt2017_A101_S101_L001_R1_001.fastq.gz > /home/qiime2/Greenhouse_experiment_2017/trimmed_16S/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

In [ ]:
#for removing ITS samples from our R2 sample (ITS4 primer)
cutadapt -a TCCTCCGCTTATTGATATGC \
--match-read-wildcards \
-m 1 \
/home/qiime2/Greenhouse_experiment_2017/180612_BVDJ2/Gh_expt2017_A101_S101_L001_R1_001.fastq.gz > /home/qiime2/Greenhouse_experiment_2017/trimmed_16S/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

because we can only do these samples one at a time we will have to write a for-loop in the shell using bash script. Also, we will run grep our test samples to see what's up. 

In [3]:
grep TCCTCCGCTTATTGATATGC -c unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R2_001.fastq

SyntaxError: invalid syntax (<ipython-input-3-e41f56baf5ef>, line 1)

In [4]:
grep CCG[C,T]CAATT[C,T][A,C]TTT[A,G]AGTT -c unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R1_001.fastq

SyntaxError: invalid syntax (<ipython-input-4-926c38af436e>, line 1)

10479 is the approx. # of reads from the "total" that contain the ITS4 forward primer. Therefore, 5299 of the reads are leftover. However, we got 2742 counts for bacteria/archea, which is werid because total is ~15000 reads.


In [ ]:
grep CCG[A-Z]CAATT[A-Z][A-Z]TTT[A-Z]AGTT -c trimmed_ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

after grep we get 10479 counts of our ITS primer sequence from the trimmed ITS sequence. Therefore, cutadapt didn't remove any ITS sequences from our fastq file.  

In [ ]:
grep CCG[A-Z]CAATT[A-Z][A-Z]TTT[A-Z]AGTTC -c trimmed_ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

after cutadapting the 16S primers, we only found **12** counts of our 926r primer. So it looks like cutadapt worked! Because its definitely a lot less than 2742 reads/counts

In [ ]:
grep CCG[A-Z]CAATT[A-Z][A-Z]TTT[A-Z]AGTT -c trimmed_16S/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq

Grep shows that in our trimmed 16S samples there are 2742 counts of the 962r primer for bacteria.

In [ ]:
grep CCG[A-Z]CAATT[A-Z][A-Z]TTT[A-Z]AGTT -c trimmed_16S/Gh_ept2017_A101_R2_001.fastq_trimmed.fastq

In [ ]:
gzip -k -r /home/qiime2/Greenhouse_experiment_2017/trimmed_16S

Here we will import our fastq.gz files to qiime2 by converting into a format that qiime2 recognizes which is .qza file. 

In [1]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /home/qiime2/Greenhouse_experiment_2017/manifestITS_Gh2017.csv \
  --output-path /home/qiime2/Greenhouse_experiment_2017/ITS/GH_2017_seq.qza \
  --source-format PairedEndFastqManifestPhred33

SyntaxError: invalid syntax (<ipython-input-1-0baa6b92f07c>, line 1)

Ran in to issues when trying to run samples through dada2, there was a mismatch error for some of our sequences according to the error message. We will first try to redo our cutadapt with a slight change:

In [ ]:
#Removing 16S sequences from our reads...
cutadapt -a CCGYCAATTYMTTTRAGTT \
-A GTGCCAGCMGCCGCGGTAA \
--match-read-wildcards \
--discard-trimmed \
-m 1 \
-o /home/qiime2/Greenhouse_experiment_2017/trimmed_ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq \
-p /home/qiime2/Greenhouse_experiment_2017/trimmed_ITS/Gh_ept2017_A101_R2_001.fastq_trimmed.fastq \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R1_001.fastq \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R2_001.fastq 

In [ ]:
#Removing ITS sequences from our reads...
cutadapt -a TCCTCCGCTTATTGATATGC \
-A GATGAAGAACGYAGYRAA \
--match-read-wildcards \
--discard-trimmed \
-m 1 \
-o /home/qiime2/Greenhouse_experiment_2017/ITS/Gh_ept2017_A101_R1_001.fastq_trimmed.fastq \
-p /home/qiime2/Greenhouse_experiment_2017/ITS/Gh_ept2017_A101_R2_001.fastq_trimmed.fastq \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R1_001.fastq \
/home/qiime2/Greenhouse_experiment_2017/unzipped_GH17-seqs/Gh_expt2017_A101_S101_L001_R2_001.fastq 

lastly, we want to remove our primers from our sequences as well

In [ ]:
qiime cutadapt trim-paired \
--i-demultiplexed-sequences /home/qiime2/Greenhouse_experiment_2017/test/GH_2017_seq.qza  \
--p-adapter-f TCCTCCGCTTATTGATATGC \ 
--p-adapter-r GATGAAGAACGYAGYRAA \
--p-match-read-wildcards \
--p-match-adapter-wildcards \
--o-trimmed-sequences /home/qiime2/Greenhouse_experiment_2017/test/Ghexpt2017_16S_trimmed.qza \
--verbose

In [ ]:
qiime cutadapt trim-paired \
--i-demultiplexed-sequences /home/qiime2/Greenhouse_experiment_2017/ITS/GH_2017_seq.qza  \
--p-adapter-f CCGYCAATTYMTTTRAGTT \ 
--p-adapter-r GTGCCAGCMGCCGCGGTAA \
--p-match-read-wildcards \
--p-match-adapter-wildcards \
--o-trimmed-sequences /home/qiime2/Greenhouse_experiment_2017/ITS/Ghexpt2017_ITS_trimmed.qza \
--verbose

Now, we will write a for-loop that incorporates cutadapt function! (Done!, please look at cutadapt_R1.sh)